# 3.2 Edge detection -- Operators based on the second derivative

There are more advanced methods for edge detection than those based on the first-derivative.  This notebook covers some second-derivative based methods ike:

- Laplace operator
- LoG operator
- Canny algorithm

We will not implement those operators because they are more complex and it may take some time. Luckily, they are already implemented in openCV (`cv2.Laplacian` and  `cv2.Canny`) and we will used them in this notebook.

## Problem context - Edge detection for medical images

Unfortunately, you were not accepted (yet!) by the researching team at *Hospital Clínico* because the obtained results in the previous notebook were not as good as expected. Anyway, they have shown you the algorithms that they are currently using so you can study it for future interviews. Let's have a look!

<img src="./images/hired.jpg" width="400">

In [6]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib
from ipywidgets import interact, fixed, widgets
matplotlib.rcParams['figure.figsize'] = (15.0, 15.0)

images_path = './images/'

### Laplace and LoG operators 



Compared with the first derivative-based edge detectors such as the Sobel operator, the Laplacian operator may yield better results in edge localization.  

#### Implementation

A first derivative is used (openCV uses Sobel but any is valid): $\\[5pt]$

$$\frac{\partial f(x,y)}{\partial x} \approx f(i+1,j) - f(i,j) \\[5pt]$$

$$\frac{\partial f(x,y)}{\partial y} \approx f(i,j+1) - f(i,j) \\[5pt]$$

Then, take second derivatives:

$$g = \frac{\partial f^2}{\partial x^2} \approx f(i+1,j) - 2f(i,j) + f(i-1,j) \\[5pt]$$

$$h = \frac{\partial f^2}{\partial y^2} \approx f(i,j+1) - 2f(i,j) + f(i,j-1) \\[10pt]$$

Finally, apply distributive property of convolution:

<img src="./images/laplace.png" width="300">

Unfortunately, although the Laplacian operator would give us as a result a 1-pixel width edge, it is very sensitive to noise! Solution: If the image is blurred using a Gaussian filter before applying the Laplace operator, we can partially solve the noise problem. In this case, it is called **LoG** (Laplace of Gaussian).


**<span style="color:red">EXERCISE 3.2.1 -- </span>** First, complete the function **gaussian_smoothing** that blur an image using a Gaussian filter. Then, normalizes it and returns the resulting image.

In [2]:
# EX_3.2.1
# Implement a function that blurres an input image using a Gaussian filter and then normalizes it.
def gaussian_smoothing(image, sigma, w_kernel):
    """ Blur and normalize input image.   
    
        Args:
            image: Input image to be blurred
            sigma: Standard deviation of the Gaussian distribution
            w_kernel: Kernel aperture size
                    
        Returns: 
            normalized: Normalized, blurred image
    """   
    # Write your code here!

Now, we are going to see the differences between the Laplace and LoG operators. 

**<span style="color:red">EXERCISE 3.2.2 -- </span>** Implement the function **laplace_testing** that applies the Laplacian operator to the input image and to a blurred version of the input image (use the previously implemented function). Display both images along with the original one in a 1x3 plot. Use as inputs: an image, the size of the Laplacian filter (should be odd), and the parameters of the gaussian filter as input.

Note that it would possible to reduce the computation time if LoG would be precomputed. This is convolving Laplace and Gaussian filters instead of applying them separatelly.

*Tip: openCV defines the Laplace operator as [cv2.Laplacian()](https://docs.opencv.org/3.4/d5/db5/tutorial_laplace_operator.html)*

In [11]:
# EX_3.2.2
# Implement a function that applies the Laplacian operator to the input image and to a blurred version of it. 
# Display a 1x3 plot with the original image and the two resulting edge images.
# Inputs: image, size of the Laplacian kernel, sigma and size of the Gaussian kernel
def laplace_testing(image, size_Laplacian, sigma, w_gaussian):
    """ Apply Laplacian and Log operators to an image.   
    
        Args:
            image: Input image to be binarized
            size_Laplacian: size of Laplacian kernel (odd)
            sigma: Standard deviation of the Gaussian distribution
            w_gaussian: Gaussian kernel aperture size
    """  
    # Write your code here!

**What to do?** Try this method and play with interactive parameters.$\\[5pt]$      
After that, **answer following questions**:

- Can be Laplacian applied without a previous blurring?
- What would be needed for obtaining the edges from those images?

**<span style="color:blue">(Answer these questions here!)</span>**

In [12]:
# Read an image
image = cv2.imread(images_path + 'medical_3.jpg', 0)

# Interact with the parameters!
interact(laplace_testing, image=fixed(image), size_Laplacian=(1,7,2), sigma=(1,3,0.1), w_gaussian=(1,3,1))

interactive(children=(IntSlider(value=3, description='size_Laplacian', max=7, min=1, step=2), FloatSlider(valu…

<function __main__.laplace_testing>

### Canny algorithm

The Canny edge detector is an algorithm that applies the DroG operator, non-maxima suppression and hysteresis for edge detection in images.

Steps:

1. Filter out any noise using a Gaussian filter.$\\[5pt]$

2. Apply the DroG operator.$\\[5pt]$

3. *Non-maximum suppression* is applied. This removes pixels that are not considered to be part of an edge. Typically, the gradient image obtained after using DroG presents *thick* edges. The idea is to keep only those pixels that are maximum within their neighborhood in the gradient direction, supressing the rest of them. Hence, only thin lines (candidate edges) will remain.$\\[5pt]$

<img src="./images/canny_nonmaxima.png" width="800">

4. *Hysteresis*: The final step. Canny uses two thresholds (upper and lower) to determine edge pixels:

   - If the grey level of a candidate pixel of the gradient image is higher than the upper threshold, the pixel is accepted as an edge. 
   - If the grey level of a candidate pixel of the gradient image is below the lower threshold, then it is rejected. 
   - If the grey level of a candidate pixel of the gradient image is between the two thresholds, then it will be accepted only if it is connected to a pixel that is above the upper threshold and rejected otherwise.
   
<img src="./images/hysteresis.png" width="800">
   
**<span style="color:red">EXERCISE 3.2.3 -- </span>** Complete **canny_testing**, which applies the Canny algorithm to an image and to that image but previously blurred *(note that openCV Canny's implementation does not apply Gaussian blurring)*. Then both are displayed along the original image. It takes an image, both lower and upper Canny thresholds, and the parameters of the Gaussian filter as input.

*Tip: openCV defines canny algorithm as [cv2.Canny()](https://docs.opencv.org/2.4/modules/imgproc/doc/feature_detection.html?highlight=canny)*

In [13]:
# EX_3.2.3
# Implement a function that applies the Canny operator to an input image and to a blurred version of it. 
# Display a 1x3 plot with the original image and the two resulting edge images.
# Inputs: image, size of the Laplacian kernel, sigma and size of the Gaussian kernel
def canny_testing(image, lower_threshold, upper_threshold, sigma, w_gaussian):
    """ Apply Canny algorithm to an image.   
    
        Args:
            image: Input image to be binarized
            lower_threshold: bottom value for hysteresis
            upper_threshold: top value for hysteresis
            sigma: Standard deviation of the Gaussian distribution
            w_gaussian: Gaussian kernel aperture size
    """  
    # Write your code here!

**What to do?** Try this method and play with interactive parameters.$\\[5pt]$      
After that, **answer following questions**:
- Can be Canny applied without a previous blurring?
- What is a *good* value for both thresholds

In [14]:
# Read an image
image = cv2.imread(images_path + 'medical_2.jpg', 0)

# Interact with the parameters
interact(canny_testing, image=fixed(image), lower_threshold=(0,260,20), upper_threshold=(0,260,20), sigma=(1,3,0.1), w_gaussian=(1,3,1))

interactive(children=(IntSlider(value=120, description='lower_threshold', max=260, step=20), IntSlider(value=1…

<function __main__.canny_testing>

## Conclusion

Terrific! You finished this notebook, that includes information about

- Laplace (and LoG) operator and the importance of smoothing.
- how to use the Canny algorithm, and how it is implemented.

**Extra**

The Canny algorithm is a very known algorithm in the field, it is used in a lot of current technologies. Although it still gets very good results, the original paper was published in 1986 by John Canny. 

If you have curiosity about the original paper, you can download it [here](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=10&ved=2ahUKEwiU9uyiganoAhWNDWMBHducCvsQFjAJegQIBhAB&url=http%3A%2F%2Fciteseerx.ist.psu.edu%2Fviewdoc%2Fdownload%3Fdoi%3D10.1.1.420.3300%26rep%3Drep1%26type%3Dpdf&usg=AOvVaw3tsKoxnc3qnS7bji3HmvQc).